In [83]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from utils import simulate_tournament, simulate_n_tournaments, compare_settings, initiate_league


New LDC format places of interest :
* The 8 first teams qualify directly to round of 16
* From 9 to 24 : pseudo round of 32
* Interesting teams : i (9th) and y(25th)  

Format :
* 36 teams,
* 8 games,
* Hat determination of the first 2 games

LoL :
* 16 teams,
* Top 8 to Quarter finales,
* 6 games (not exactly, teams with 3 win qualifies, team with 3 loss gets eliminated)


In [61]:
from utils import assign_opponents, play_round
n_teams = 8
n_games = 4


In [69]:
pgm, lt = initiate_league(n_teams,n_games)
for i in range(n_games-1):
    pgm, lt = assign_opponents(pgm,lt,i+1, verbose = True)
    lt = play_round(lt,i+1, verbose = False)


Successfull allocation based on ranking
Successfull allocation based on ranking
Successfull allocation based on ranking


In [70]:
lt


,Level,Strategy,Nb_win,Nb_games,Win_rate,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result
a,1.0,[],3,3,1.0,c,Win,b,Win,e,Win,-,-
e,0.5,[],2,3,0.666667,f,Win,d,Win,a,Loss,-,-
d,0.625,[],2,3,0.666667,g,Win,e,Loss,b,Win,-,-
c,0.75,[],2,3,0.666667,a,Loss,h,Win,f,Win,-,-
b,0.875,[],1,3,0.333333,h,Win,a,Loss,d,Loss,-,-
f,0.375,[],1,3,0.333333,e,Loss,g,Win,c,Loss,-,-
h,0.125,[],1,3,0.333333,b,Loss,c,Loss,g,Win,-,-
g,0.25,[],0,3,0.0,d,Loss,f,Loss,h,Loss,-,-


In [71]:
pgm


,a,b,c,d,e,f,g,h
a,False,False,False,True,False,True,True,True
b,False,False,True,False,True,True,True,False
c,False,True,False,True,True,False,True,False
d,True,False,True,False,False,True,False,True
e,False,True,True,False,False,False,True,True
f,True,True,False,True,False,False,False,True
g,True,True,True,False,True,False,False,False
h,True,False,False,True,True,True,False,False


In [82]:
team = 'c'

tmp = lt[["Win_rate"]]
tmp = tmp.merge(pgm[team], left_index = True, right_index = True)
tmp[tmp[team]].index


Index(['e', 'd', 'b', 'g'], dtype='object')

In [168]:

pairs = []
delta_wr = []

for a in lt.index :
    for b in lt.index :
        if pgm.loc[a,b]:
            delta_wr.append([a,b,np.abs(lt.loc[a,'Win_rate'] - lt.loc[b,'Win_rate'])])

pairs_delta_wr = pd.DataFrame(delta_wr, columns = ['team_1', 'team_2', 'delta_wr'])

pairs_delta_wr.head()


,team_1,team_2,delta_wr
0,a,d,0.333333
1,a,f,0.666667
2,a,h,0.666667
3,a,g,1.000000
4,e,c,0.000000


In [215]:
n_teams = len(list(lt.index))
pairs = [[a,b] for a,b in zip(pairs_delta_wr['team_1'],pairs_delta_wr['team_2'])]
#pairings = [[pair] for pair in pairs]
pairings = [[pair] for pair in pairs]

pairs[0][0] in pairings[0][0]


True

In [213]:
pairs[0]


['a', 'd']

In [216]:
tmp = pairings[0] + [pairs[0]]
tmp


[['a', 'd'], ['a', 'd']]

In [217]:

for i in range(int(n_teams/2)-1):
    pps = []
    print(f'Iteration number {i+1}')
    for pairing in pairings :
        for pair in pairs :
            possible = True
            for team in pair :
                for game in pairing :
                    if team in game :
                        possible = False
            if possible :
                new_pairing = pairing + [pair]
                pps.append(new_pairing) 
    pairings = pps.copy()  

len(pairings)


Iteration number 1
Iteration number 2
Iteration number 3


6144

In [218]:
pairings


[[['a', 'd'], ['e', 'c'], ['b', 'g'], ['f', 'h']],
 [['a', 'd'], ['e', 'c'], ['b', 'g'], ['h', 'f']],
 [['a', 'd'], ['e', 'c'], ['f', 'h'], ['b', 'g']],
 [['a', 'd'], ['e', 'c'], ['f', 'h'], ['g', 'b']],
 [['a', 'd'], ['e', 'c'], ['h', 'f'], ['b', 'g']],
 [['a', 'd'], ['e', 'c'], ['h', 'f'], ['g', 'b']],
 [['a', 'd'], ['e', 'c'], ['g', 'b'], ['f', 'h']],
 [['a', 'd'], ['e', 'c'], ['g', 'b'], ['h', 'f']],
 [['a', 'd'], ['e', 'b'], ['c', 'g'], ['f', 'h']],
 [['a', 'd'], ['e', 'b'], ['c', 'g'], ['h', 'f']],
 [['a', 'd'], ['e', 'b'], ['f', 'h'], ['c', 'g']],
 [['a', 'd'], ['e', 'b'], ['f', 'h'], ['g', 'c']],
 [['a', 'd'], ['e', 'b'], ['h', 'f'], ['c', 'g']],
 [['a', 'd'], ['e', 'b'], ['h', 'f'], ['g', 'c']],
 [['a', 'd'], ['e', 'b'], ['g', 'c'], ['f', 'h']],
 [['a', 'd'], ['e', 'b'], ['g', 'c'], ['h', 'f']],
 [['a', 'd'], ['e', 'h'], ['c', 'g'], ['b', 'f']],
 [['a', 'd'], ['e', 'h'], ['c', 'g'], ['f', 'b']],
 [['a', 'd'], ['e', 'h'], ['b', 'f'], ['c', 'g']],
 [['a', 'd'], ['e', 'h'], ['b',

In [ ]:
pairs_delta_wr.loc[game,'delta_wr']


In [142]:
deltas_wr = []
for pp in pairings :
    delta_wr = 0
    for game in pp :
        delta_wr += pairs_delta_wr.loc['(e,b)','delta_wr']
    deltas_wr.append(delta_wr)
    
pd.DataFrame(deltas_wr, index=pairings, columns=['delta_WR'])


KeyError: '(e,b)'

In [56]:
def assign_opponents(possible_games_matrix, league_table, round_number, verbose = False):
    
    if f"R{round_number}_opponent" not in league_table.columns: 
        raise ValueError(f'The league was initiated with less than {round_number} games')
    
    ref_str = f'R{round_number}_opponent'
    
    lt = league_table.copy()
    pgm = possible_games_matrix.copy()
    
    nb_success = 0
    if round_number == 1 :
        lt = lt.sample(frac = 1)
    else :
        lt.sort_values(by='Win_rate', ascending = False, inplace=True)
    
    # Preferred allocation based on rankings from top to bottom
    for team in lt.index:
        if lt.loc[team, ref_str] == '-' :
            for opposing_team in lt.index:
                if opposing_team != team and pgm.loc[team,opposing_team] and lt.loc[opposing_team, ref_str] == '-' :
                    lt.loc[team, ref_str] = opposing_team
                    lt.loc[opposing_team, ref_str] = team
                    pgm.loc[team, opposing_team] = False
                    pgm.loc[opposing_team, team] = False
                    nb_success +=2
                    if verbose and nb_success == lt.shape[0] :
                        print('Successfull allocation based on ranking')
                    break
    
    
    
    # Last resort : random allocation if none of the above worked
    if not nb_success == lt.shape[0] :
        keep_going = True
        while keep_going :
            nb_success = 0
            lt = league_table.copy()
            pgm = possible_games_matrix.copy()           
            lt = lt.sample(frac = 1)
            for team in lt.index:
                if lt.loc[team, ref_str] == '-' :
                    for opposing_team in lt.index:
                        if opposing_team != team and pgm.loc[team,opposing_team] and lt.loc[opposing_team, ref_str] == '-' :
                            lt.loc[team, ref_str] = opposing_team
                            lt.loc[opposing_team, ref_str] = team
                            pgm.loc[team, opposing_team] = False
                            pgm.loc[opposing_team, team] = False
                            nb_success +=2
                            if nb_success == lt.shape[0]:
                                keep_going = False
                            break     
        if verbose :
            print('Unsuccessfull allocation based on rankings, random allocation was performed')                          
    return pgm, lt


Successfull allocation based on ranking
Round number 1, game g vs e of levels 0.02 vs 0.06: victory for e 
Round number 1, game c vs d of levels 0.25 vs 0.12: victory for c 
Round number 1, game f vs b of levels 0.03 vs 0.5: victory for b 
Round number 1, game h vs a of levels 0.01 vs 1.0: victory for a 
Successfull allocation based on ranking
Round number 2, game e vs c of levels 0.06 vs 0.25: victory for c 
Round number 2, game b vs a of levels 0.5 vs 1.0: victory for a 
Round number 2, game g vs d of levels 0.02 vs 0.12: victory for d 
Round number 2, game f vs h of levels 0.03 vs 0.01: victory for f 
Successfull allocation based on ranking
Round number 3, game c vs a of levels 0.25 vs 1.0: victory for a 
Round number 3, game e vs b of levels 0.06 vs 0.5: victory for b 
Round number 3, game d vs f of levels 0.12 vs 0.03: victory for f - UNEXPECTED 
Round number 3, game g vs h of levels 0.02 vs 0.01: victory for h - UNEXPECTED 
Unsuccessfull allocation based on rankings, random alloc

,Level,Strategy,Nb_win,Nb_games,Win_rate,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result
a,1.0,[],4,4,1.0,h,Win,b,Win,c,Win,g,Win
b,0.5,[],3,4,0.75,f,Win,a,Loss,e,Win,h,Win
c,0.25,[],3,4,0.75,d,Win,e,Win,a,Loss,f,Win
d,0.125,[],2,4,0.5,c,Loss,g,Win,f,Loss,e,Win
f,0.03125,[],2,4,0.5,b,Loss,h,Win,d,Win,c,Loss
h,0.007812,[],1,4,0.25,a,Loss,f,Loss,g,Win,b,Loss
e,0.0625,[],1,4,0.25,g,Win,c,Loss,b,Loss,d,Loss
g,0.015625,[],0,4,0.0,e,Loss,d,Loss,h,Loss,a,Loss


In [7]:
compare_settings(n_tournaments=1000, 
                 n_teams=16, 
                 n_rounds = 6, 
                 delta_level ='exponential',
                 strategies={'i':[1]}, 
                 probabilistic=True, 
                 deterministic=True)


Probabilistic resolution


,Level,Strategy,Strategic_avg_WR,Control_avg_WR,Delta
a,1.0,[],0.911833,0.917833,-0.006
b,0.5,[],0.837833,0.830333,0.0075
c,0.25,[],0.7735,0.769667,0.003833
d,0.125,[],0.714167,0.712667,0.0015
e,0.0625,[],0.66,0.661833,-0.001833
f,0.03125,[],0.613833,0.613,0.000833
g,0.015625,[],0.567,0.568167,-0.001167
h,0.007812,[],0.5305,0.526333,0.004167
i,0.003906,[1],0.457667,0.482333,-0.024667
j,0.001953,[],0.434333,0.43,0.004333


---------------------------------------------------
Deterministic resolution


,Level,Strategy,Strategic_avg_WR,Control_avg_WR,Delta
a,1.0,[],1.0,1.0,0.0
b,0.5,[],0.833333,0.833333,0.0
c,0.25,[],0.773167,0.769,0.004167
d,0.125,[],0.719,0.715,0.004
e,0.0625,[],0.668333,0.668,0.000333
f,0.03125,[],0.6185,0.616,0.0025
g,0.015625,[],0.559,0.565167,-0.006167
h,0.007812,[],0.523667,0.520167,0.0035
i,0.003906,[1],0.465833,0.476333,-0.0105
j,0.001953,[],0.433167,0.4345,-0.001333
